In [1]:
import pexpect
import itertools

In [2]:
def spawn_jdb(classname, child=None):
    try:
        child.close()
    except:
        pass

    child = pexpect.spawnu('jdb {classname}'.format(
        classname=classname))
    child.sendline("stop in {classname}.main".format(
        classname=classname))
    child.expect(".*Deferring breakpoint.*")
    child.sendline("run")
    child.expect(".*Breakpoint hit:")
    
    return child

In [3]:
def parse_rawstr(rawstr):
    # Remove /r
    rawstr = rawstr.replace('\r\n', '\n')
    
    # Strip trailing spaces
    rawstr = rawstr.strip()
    
    # Split into lines
    lines = rawstr.splitlines()
    
    # Parse each line
    variables = []
    for line in lines:
        if '=' not in line:
            continue
        parts = line.split(sep='=', maxsplit=1)
        name = parts[0].strip()
        value = parts[1].strip()
        variables.append((name, value))
    
    return variables

In [4]:
def step(child):
    if child.closed or child.eof():
        return None
        
    # Make a step
    child.sendline("step in")
    
    # Print out all local variables
    child.sendline("locals")
    
    # Expect the variables from method arguments
    child.expect("Method arguments:")
    
    # Expect the variables locally declared
    child.expect("Local variables:")
    
    rawstr_args = child.before
    
    # Seek forward to prompt
    child.expect(".*\[.*\] ")
    
    rawstr_locals = child.after
    
    # Parse the strings
    args = parse_rawstr(rawstr_args)
    local = parse_rawstr(rawstr_locals)
    
    return (args, local)

In [5]:
def dump(child, obj):
    
    # First get a prompt
    child.sendline("")
    child.expect(".*\[.*\] ")
    rawstr_prompt =  child.after.strip()
    
    # Escape prompt
    esc_prompt = rawstr_prompt
    esc_prompt = esc_prompt.replace("[", "\[")
    esc_prompt = esc_prompt.replace("]", "\]")
    
    # Send command
    child.sendline("dump {}".format(obj))
    
    # Expect output
    pattern = "{obj} = .*{prompt}".format(
        obj=obj,
        prompt=esc_prompt)
    child.expect(pattern)
    
    # Parse output
    ret = child.after
    
    # - remove last line break and prompt that follows
    last_line_break = ret.rfind("\n")
    ret = ret[:last_line_break].strip()
    
    # - remove obj name and equal sign
    ret = ret.replace("{} =".format(obj), "").strip()
    
    if ret == "null":
        ret = None
    
    return ret

In [6]:
variables = {}

child = spawn_jdb("IterPower")

while True:
    ret = None
    try:
        ret = step(child)
    except pexpect.EOF:
        break
    if ret is None:
        continue
        
    #
    (args, locs) = ret
    for (var, val) in itertools.chain(args, locs):
        variables[var] = variables.get(var, list())
        variables[var].append(val)

variables_unique_values = { var: set(vals) for (var, vals) in variables.items() }

In [7]:
variables_unique_values

{'base': {'10'},
 'exp': {'0', '1', '2', '3', '4'},
 'result': {'1', '10', '100', '1000', '10000'},
 'args': {'instance of java.lang.String[0] (id=495)'}}